In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
raw_train_df = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv') 
raw_test_df = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')

In [ ]:
raw_train_df.tail()

In [ ]:
full_df = pd.concat([raw_train_df,raw_test_df])

In [ ]:
full_df.drop('id',axis=1,inplace=True)

In [ ]:
full_df.describe()

In [ ]:
full_df.isnull().any()

In [ ]:
full_df.cat0 = full_df.cat0.replace({'A':0,'B':1})
full_df.cat1 = full_df.cat1.replace({'A':0,'B':1})
full_df.cat2 = full_df.cat2.replace({'A':0,'B':1})
full_df.cat3 = full_df.cat3.replace({'A':0,'B':1,'C':3,'D':4})
full_df.cat4 = full_df.cat4.replace({'A':0,'B':1,'C':3,'D':4})
full_df.cat5 = full_df.cat5.replace({'A':0,'B':1,'C':3,'D':4})
full_df.cat6 = full_df.cat6.replace({'A':0,'B':1,'C':2,'D':3,'E':4,'I':5,'G':6,'H':7})
full_df.cat7 = full_df.cat7.replace({'A':0,'B':1,'C':2,'D':3,'E':4,'I':5,'G':6,'F':7})
full_df.cat8 = full_df.cat8.replace({'A':0,'B':1,'C':2,'D':3,'E':4,'G':5,'F':6})
full_df.cat9 = full_df.cat9.replace({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,'N':13,'O':14})

In [ ]:
df = full_df.iloc[:300000][:]

In [ ]:
for i in df.columns[10:24]:
    outlier1 = df[i].quantile(0.98)
    outlier2 = df[i].quantile(0.02)
    train_df = df[(df[i]<outlier1) & (df[i]>outlier2)]

In [ ]:
test_df = full_df[300000:][:]
test_df.drop('target',axis=1,inplace=True)

In [ ]:
test_df

In [ ]:
X = train_df.drop('target',axis=1)

In [ ]:
y = train_df.pop('target')

In [ ]:
X_train, y_train, X_valid, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#y_train = scaler.fit_transform(y_train)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(2048,activation='relu',kernel_initializer='normal', input_dim=24))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1,activation='relu'))

In [ ]:
model.compile(
    loss='mae',optimizer=tf.keras.optimizers.SGD(
   learning_rate=0.5)
)
model.summary()

In [ ]:
model.fit(X_train,X_valid,epochs=30,validation_data=(y_train,y_valid))

In [ ]:
XGmodel = XGBRegressor()

In [ ]:
logistic = LinearRegression()

In [ ]:
logistic

In [ ]:
XGmodel.fit(X_train,X_valid)

In [ ]:
logistic.fit(X_train.values, X_valid.values)

In [ ]:
#val_preds = XGmodel.predict(y_train)
val_preds2 = logistic.predict(y_train)
#rmse = mean_squared_error(y_valid, val_preds, squared=True)
rmse = mean_squared_error(y_valid, val_preds2, squared=True)

In [ ]:
X_train

In [ ]:
rmse

In [ ]:
target = logistic.predict(test_df)

In [ ]:
target = target.reshape(200000,1)

In [ ]:
raw_test_df['target'] = target

In [ ]:
submission = raw_test_df.drop(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5',
       'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12',
       'cont13'],axis=1,)

In [ ]:
submission.set_index('id',inplace=True)

In [ ]:
submission.to_csv('./submission.csv')